## Import the necessary packages

Set the matplotlib backend so figures can be saved in the background

In [1]:
import matplotlib
matplotlib.use("Agg")

In [2]:
import sys
sys.path.append("..")
import os
os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin")
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from nn.conv.minivggnet import MiniVGGNet
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np

Construct the dictionary instead of argument parse

In [3]:
args = {
    "output" : "output/cifar10_minivggnet_with_bn.png"
}

Load the training and testing data, then scale it into the range [0, 1]

In [4]:
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

[INFO] loading CIFAR-10 data...


Convert the labels from integers to vectors

In [5]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.fit_transform(testY)

Initialize the label names for the CIFAR-10 dataset

In [6]:
labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

## Let's compile our model and start training MiniVGGNet:

Initialize the optimizer and model

In [7]:
print("[INFO] compiling model...")
opt = SGD(lr=0.01, decay=0.01/40, momentum=0.9, nesterov=True)
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


d:\tu hoc\deep learning for computer vision with python\starter_bundle\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Train the network

In [10]:
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=40, verbose=1)

[INFO] training network...
Epoch 1/40
115/782 [===>..........................] - ETA: 15s - loss: 0.8826 - accuracy: 0.6887

KeyboardInterrupt: 

Evaluate the network

In [11]:
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))

[INFO] evaluating network...
              precision    recall  f1-score   support

    airplane       0.76      0.71      0.73      1000
  automobile       0.79      0.86      0.83      1000
        bird       0.74      0.39      0.51      1000
         cat       0.47      0.54      0.50      1000
        deer       0.68      0.64      0.66      1000
         dog       0.59      0.58      0.58      1000
        frog       0.70      0.86      0.77      1000
       horse       0.81      0.72      0.76      1000
        ship       0.77      0.87      0.82      1000
       truck       0.71      0.84      0.77      1000

    accuracy                           0.70     10000
   macro avg       0.70      0.70      0.69     10000
weighted avg       0.70      0.70      0.69     10000



Plot the training loss and accuracy

In [ ]:
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 40), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 40), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 40), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, 40), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on CIFAR-10")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(args["output"])